In [2]:
import psycopg2
import nltk
import pandas as pd
import numpy as np
import re
import string
import spacy
import sklearn
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [3]:
# importing and creating df (has to have type_id)

# usage: specify file location, sample size and seed(used by random)
#filepath = '../Data_sample/FakeNewsCorpus_250.csv' # 250 rows of FakeNewsCorpus
filepath = '/Users/Master/Documents/KU/2.Semester/Datascience/1mioraw.csv'       # 1 mil rows raw
s = 1000000                                            # desired sample size
seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_csv(filepath, index_col = [0]).sample(n=s, random_state=seed)
df["content"] = df["content"].astype(str)
# create type_id
df['type_id'] = df.groupby(['type']).ngroup()

df.head()

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,type_id
6803,322758,zerohedge.com,conspiracy,https://www.zerohedge.com/news/2014-12-17/new-...,Submitted by Mike Krieger via Liberty Blitzkri...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,59% Of Americans Support Post-9/11 Torture – P...,NaN,NaN,[''],"""By an almost 2-1 margin, or 59-to-31 percent,...",NaN,NaN,NaN,2
9359,1017129,shadowproof.com,unknown,https://shadowproof.com/2012/09/18/oregons-mar...,"Measure 80, which would legalize marijuana for...",2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Oregon’s Marijuana Legalization Measure Traili...,"Jon Walker, Jonathan Walker Grew Up In New Jer...",NaN,[''],NaN,NaN,NaN,NaN,10
4476,577340,us.blastingnews.com,satire,http://us.blastingnews.com/news/2017/05/photo/...,This website uses profiling (non technical) co...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Photogallery - Donald Trump 'gaining weight' o...,"Blasting News, P. Ghose, M. Singh, W. Camille,...",NaN,"['Donald Trump', 'Russian Scandal', 'Gaining W...",Article's photos Donald Trump 'gaining weight'...,"4062 followers Duggar Family, 3756 followers S...",NaN,NaN,9
988,695601,breitbart.com,political,http://www.breitbart.com/author/joseph-c-phill...,It is said that when Alexander the Great visit...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"Joseph C. Phillips, Author at Breitbart",Joseph C. Phillips,NaN,[''],"Joseph C. Phillips, Author at Breitbart - Page...",NaN,NaN,NaN,6
155,1150037,express.co.uk,rumor,https://www.express.co.uk/sport/football/61716...,"Barca, along with Arsenal and Liverpool, have ...",2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Arsenal and Liverpool target to snub bargain £...,Joe Short,NaN,[''],EZEQUIEL LAVEZZI will not be joining Barcelona...,Wenger would likely be tempted by a cheap deal...,NaN,NaN,8


In [14]:
df['id'] = pd.to_numeric(df['id'], errors = 'coerce', downcast = 'integer')
df.drop_duplicates(subset = 'id', inplace = True)
df = df.dropna(subset=['id'])


In [16]:
df.id.astype(int)

6803     322758
9359    1017129
4476     577340
988      695601
155     1150037
         ...   
1240     561926
1558     952987
901      534794
1732     562502
8026     146392
Name: id, Length: 999934, dtype: int64

In [17]:
df.head()

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,type_id
6803,322758,zerohedge.com,conspiracy,https://www.zerohedge.com/news/2014-12-17/new-...,Submitted by Mike Krieger via Liberty Blitzkri...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,59% Of Americans Support Post-9/11 Torture – P...,NaN,NaN,[''],"""By an almost 2-1 margin, or 59-to-31 percent,...",NaN,NaN,NaN,2
9359,1017129,shadowproof.com,unknown,https://shadowproof.com/2012/09/18/oregons-mar...,"Measure 80, which would legalize marijuana for...",2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Oregon’s Marijuana Legalization Measure Traili...,"Jon Walker, Jonathan Walker Grew Up In New Jer...",NaN,[''],NaN,NaN,NaN,NaN,10
4476,577340,us.blastingnews.com,satire,http://us.blastingnews.com/news/2017/05/photo/...,This website uses profiling (non technical) co...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Photogallery - Donald Trump 'gaining weight' o...,"Blasting News, P. Ghose, M. Singh, W. Camille,...",NaN,"['Donald Trump', 'Russian Scandal', 'Gaining W...",Article's photos Donald Trump 'gaining weight'...,"4062 followers Duggar Family, 3756 followers S...",NaN,NaN,9
988,695601,breitbart.com,political,http://www.breitbart.com/author/joseph-c-phill...,It is said that when Alexander the Great visit...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"Joseph C. Phillips, Author at Breitbart",Joseph C. Phillips,NaN,[''],"Joseph C. Phillips, Author at Breitbart - Page...",NaN,NaN,NaN,6
155,1150037,express.co.uk,rumor,https://www.express.co.uk/sport/football/61716...,"Barca, along with Arsenal and Liverpool, have ...",2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Arsenal and Liverpool target to snub bargain £...,Joe Short,NaN,[''],EZEQUIEL LAVEZZI will not be joining Barcelona...,Wenger would likely be tempted by a cheap deal...,NaN,NaN,8


In [190]:
# This can generate a dataset with random purmutation and a max size for each type(can be smaller if desired max is not possible)

# max size for type
max_size = 5000
# traning_set ratio - splits data into traning=ratio,  test and validate=(1-ratio)/2 ex. train=80%, test=10%, validate=10%
ratio=0.7
# Labels to include - ['fake', 'satire', 'bias', 'conspiracy', 'state', 'junksci', 'hate', 'clickbait', 'unreliable', 'political', 'reliable'] - all labels
use_types = ['fake', 'satire', 'bias', 'conspiracy', 'junksci', 'hate', 'clickbait', 'unreliable', 'political', 'reliable']
# Random seed
rnd = 1

# initialize dataframes
train    = pd.DataFrame(columns = df.columns)
test     = pd.DataFrame(columns = df.columns)
validate = pd.DataFrame(columns = df.columns)

# add type to test splits
for t in use_types:

    # type size
    type_size = df['type'].loc[df['type'] == t].value_counts().min()

    # set size of type slice
    if type_size < max_size:
        tmp = df.loc[df['type'] == t].sample(n = type_size, random_state=rnd)
    else:
        tmp = df.loc[df['type'] == t].sample(n = max_size, random_state=rnd)

    # split current type
    train_tmp, test_tmp, validate_tmp = np.split(tmp, [int(ratio * len(tmp)), int(((1-ratio)/2 + ratio) * len(tmp))])

    # add tmp to dataframes
    train    = pd.concat([train, train_tmp])
    test     = pd.concat([test, test_tmp])
    validate = pd.concat([validate, validate_tmp])
    
    # print split shape
    print("=>", t, tmp.shape, train_tmp.shape, validate_tmp.shape, validate_tmp.shape)

print("\n[Final split]\ntrain, test, validate ==>", train.shape, test.shape, validate.shape)

=> fake (5000, 17) (3500, 17) (750, 17) (750, 17)
=> satire (5000, 17) (3500, 17) (750, 17) (750, 17)
=> bias (5000, 17) (3500, 17) (750, 17) (750, 17)
=> conspiracy (5000, 17) (3500, 17) (750, 17) (750, 17)
=> junksci (5000, 17) (3500, 17) (750, 17) (750, 17)
=> hate (3619, 17) (2533, 17) (543, 17) (543, 17)
=> clickbait (5000, 17) (3500, 17) (750, 17) (750, 17)
=> unreliable (5000, 17) (3500, 17) (750, 17) (750, 17)
=> political (5000, 17) (3500, 17) (750, 17) (750, 17)
=> reliable (5000, 17) (3500, 17) (750, 17) (750, 17)

[Final split]
train, test, validate ==> (34033, 17) (7293, 17) (7293, 17)


In [87]:
# importing and creating df (has to have type_id)

# usage: specify file location, sample size and seed(used by random) 
filepath = '/Users/Master/Documents/KU/2.Semester/Datascience/news_sample.csv' #
#s = 250                                            # desired sample size
#seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_csv(filepath, index_col = [0])
content = df['content']

In [27]:
nlp = spacy.load('en_core_web_lg')

In [23]:
text = ("these word are features")
with nlp.disable_pipes():
    vectors = np.array([token.vector for token in nlp(text)])

In [7]:
doc = nlp("these word, are features")
for token in doc:
    print(token)
#token.lemma_
#token.is_stop

these
word
,
are
features


In [4]:
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(sentence).vector for sentence in df['content']])

In [66]:
fakeidx = df.index[df.type == 'fake'].tolist()

In [11]:
fakevecs = np.array([doc_vectors[i] for i in fakeidx])

In [54]:

len(fakevecs)
np.linalg.norm(fakevecs[0])
#calc avg vector of the type-vectors:
#np.linalg.norm(fakevecs[29])

2.442973

In [19]:
train.type.fillna('no type', inplace = True)
#X_train, X_test, y_train, y_test = train_test_split(doc_vectors, df.type, test_size = 0.1, random_state= 1)

In [28]:
a = nlp('REPLY NOW FOR FREE TEA').vector
b = nlp('REPLY NOW FOR FREE TEA AND Potatoes').vector

In [16]:
svc = LinearSVC(random_state = 1, dual = False, max_iter = 10000)
svc.fit(X_train, y_train)
print(f"Accuracy: {svc.score(X_test, y_test) * 100:.3f}%",)
#svc.score(X_test,y_test) * 100

Accuracy: 64.000%


In [29]:
#sklearn.metrics.pairwise.cosine_similarity(a,b)
def cosine_similarity(a,b):
    return a.dot(b)/np.sqrt(a.dot(a) * b.dot(b))

In [37]:
cosine_similarity(fakevecs[152],doc_vectors[45])

0.95132565

In [7]:
#try tf-idf svm
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df["content"])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x, df.type, test_size = 0.1, random_state= 1)

In [12]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.52

In [80]:
x = nlp("man")
y = nlp("king")
print(x.similarity(y))
print(x.similarity(x))



0.40884613016820903
1.0


In [191]:
import gensim 
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models.doc2vec import TaggedDocument, Doc2Vec 
from collections import namedtuple
q = word_tokenize('bag is up')
q

['bag', 'is', 'up']

In [192]:
train.type.fillna('no type', inplace = True)
train['type_id'] = train.groupby(['type']).ngroup()
train.set_index('id', inplace = True)
train.reset_index(inplace = True)

In [194]:
%%time
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
series_content = train.content

contTok = series_content.apply(nltk.word_tokenize)
typeLst =train.type_id.tolist()

CPU times: user 2min 57s, sys: 6.51 s, total: 3min 3s
Wall time: 3min 28s


In [31]:
contTok
#model = Doc2Vec(docs, size = 100, window = 300, min_count = 1, workers = 4)

id
449029    [How, To, Easily, Understand, The, Difference,...
603573    [Butter, Chocolate, Cheese, Cake, %, of, reade...
34728     [Chipotle, E., coli, O26, Outbreak, Update, –,...
337055    [U.S., Repeals, Propaganda, Ban, ,, Spreads, G...
771816    [Darwin, and, the, Voyage, :, 11, ~, Elephants...
                                ...                        
933115    [(, Photo, :, Screengrab/ERLC, ), Russell, Moo...
788062    [(, Photo, :, Screencap/First, Baptist, Dallas...
740245    [On, Monday, ,, North, Carolina, sophomore, po...
930566    [Photo, :, (, YouTube, Screenshot/Timothy, W.,...
832948    [In, 1968, ,, the, public, anger, over, the, V...
Name: content, Length: 1400, dtype: object

In [197]:
#model = gensim.models.doc2vec.Doc2Vec(vector_size=250, min_count=2, epochs=40)
#documents = [TaggedDocument(contTok,df.index) for sentence in df]
documents = [TaggedDocument(contTok, [train.index[i]]) for i, contTok in enumerate(contTok)]

In [73]:
#documents

In [74]:
contTok

id
449029    [How, To, Easily, Understand, The, Difference,...
603573    [Butter, Chocolate, Cheese, Cake, %, of, reade...
34728     [Chipotle, E., coli, O26, Outbreak, Update, –,...
337055    [U.S., Repeals, Propaganda, Ban, ,, Spreads, G...
771816    [Darwin, and, the, Voyage, :, 11, ~, Elephants...
                                ...                        
933115    [(, Photo, :, Screengrab/ERLC, ), Russell, Moo...
788062    [(, Photo, :, Screencap/First, Baptist, Dallas...
740245    [On, Monday, ,, North, Carolina, sophomore, po...
930566    [Photo, :, (, YouTube, Screenshot/Timothy, W.,...
832948    [In, 1968, ,, the, public, anger, over, the, V...
Name: content, Length: 1400, dtype: object

In [200]:
cores = multiprocessing.cpu_count()

In [201]:
%%time
model = Doc2Vec(documents, vector_size = 150, min_count = 2, window = 10, epochs= 5, workers = cores, alpha = 0.025, min_alpha = 0.001)

CPU times: user 9min 55s, sys: 13.8 s, total: 10min 9s
Wall time: 5min 10s


In [257]:
model.most_similar('potato')

[('salad', 0.825251579284668),
 ('cheese', 0.8202081322669983),
 ('lettuce', 0.8176130652427673),
 ('sauce', 0.8084850907325745),
 ('flour', 0.8066356778144836),
 ('sausage', 0.804688572883606),
 ('potatoes', 0.79893958568573),
 ('sweet', 0.794294536113739),
 ('soup', 0.7887793183326721),
 ('wine', 0.7831469178199768)]

In [258]:
print((model.docvecs.count))

34033


In [22]:
#vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
#doc2 = nlp(df.content[20])v
#tokdoc2 = [token for token in nlp(df.content[20])] 
#for token in doc2:
#    print(token)

In [67]:
#model.docvecs.most_similar([2]) unreliable

In [19]:
#train.type[(20)]

In [226]:
sim_doc = model.docvecs.most_similar([30536])
sim_doc

[(33237, 0.8410108089447021),
 (30508, 0.8348163366317749),
 (32362, 0.8327586650848389),
 (32968, 0.8319017887115479),
 (18620, 0.8270764350891113),
 (31936, 0.8258225917816162),
 (32952, 0.8226457834243774),
 (8898, 0.8212412595748901),
 (32392, 0.820825457572937),
 (19785, 0.8205291032791138)]

In [230]:
train.loc[train.index == 32362 ].type

32362    reliable
Name: type, dtype: object

In [231]:
#train.loc[train['type'] == 'reliable']

In [232]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import utils

from tqdm import tqdm
import multiprocessing


In [243]:
import numpy as np

In [320]:
#Put all docvecs and labels into arrays for classifier:
train_arrays = np.zeros((model.docvecs.count,150))
train_labels = np.zeros(model.docvecs.count)
for i in range(model.docvecs.count):
    train_arrays[i] = model[i]
    train_labels[i] = train.type_id[i]

In [313]:
train_arrays[0]
len(train_labels)
#train.loc[train.index == 20000]

34033

In [321]:
from sklearn.utils import shuffle
train_arrays, train_labels = shuffle(train_arrays, train_labels)


In [322]:
X_train, X_test, y_train, y_test = train_test_split(train_arrays_shuff, train_labels_shuff, test_size = 0.2, random_state= 2)

In [291]:
#len(train_arrays)
#train_labels

array([3., 3., 3., ..., 7., 7., 7.])

In [250]:
len(test_arrays)
test_labels

array([5., 5., 5., ..., 7., 7., 7.])

In [326]:
#Now train a logistic classifier:
classifier = LogisticRegression()
classifier.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [327]:
classifier.score(X_test,y_test)

0.2908770383428823

In [328]:
classifier.predict(X_test)

array([4., 8., 9., ..., 7., 7., 5.])

In [332]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)
#clf.score(X_test, y_test)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [333]:
clf.score(X_test, y_test)

0.5332745702952842

In [ ]:
#try tf-idf svm
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df["content"])

In [331]:
model.most_similar('potato')

[('salad', 0.825251579284668),
 ('cheese', 0.8202081322669983),
 ('lettuce', 0.8176130652427673),
 ('sauce', 0.8084850907325745),
 ('flour', 0.8066356778144836),
 ('sausage', 0.804688572883606),
 ('potatoes', 0.79893958568573),
 ('sweet', 0.794294536113739),
 ('soup', 0.7887793183326721),
 ('wine', 0.7831469178199768)]